In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse

In [2]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

In [3]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

In [4]:
def snum(a):
    if a > 0 :
        return '0'
    else :
        return '1'

In [5]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=6, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=4, help='Number of bit width')
args = parser.parse_args(args=[])

In [6]:
class	fxp:
	def	__init__(self, bIn, iBWF):
		self.iFullBW	= len(bIn)
		self.iIntgBW	= self.iFullBW - iBWF
		self.bSign		= bIn[0]
		self.bIntg		= bIn[:self.iIntgBW]
		self.bFrac		= bIn[self.iIntgBW:]
		self.fFull		= 0
		try:
			for idx, bit in enumerate(bIn):
				if	idx == 0:
					self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
				else:
					self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
		except:
			print(bIn)
		self.dispFull	= self.bIntg +"."+ self.bFrac 
		return

In [7]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [8]:
class LFSR:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[2]),int(stream[3]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3

In [9]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

In [10]:
def perm(a):
    al = len(a)
    blist = []
    for i in range(al) :
        #print(al-i-1)
        blist.append(a[al-i-1])
    
    b = "".join(blist)
    
    return b

In [11]:
def findActMaxMin():
    SF = {}
    for i in range(3):
            max = torch.max(eval(f'act{i}'))
            min = torch.min(eval(f'act{i}'))
            exec(f"SF['act{i}']=torch.max(abs(max),abs(min)).item()")
    return SF

In [12]:
def findWeiMaxMin():
    SF = {}
    for i in range(3):
            max = torch.max(eval(f'wei{i}'))
            min = torch.min(eval(f'wei{i}'))
            exec(f"SF['wei{i}']=torch.max(abs(max),abs(min)).item()")
    return SF

In [13]:
def LFSRlist():
    lfsr = LFSR()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

In [14]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
        
    oAnum = oAlist.count('1')
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    return sA

In [15]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = perm(lfsr[k])
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
        
    oAnum = oAlist.count('1')
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    return sA

In [16]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

In [17]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

In [117]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if a[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

In [118]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

In [119]:
act0 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act1 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act2 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))

wei0 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei1 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei2 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))

In [120]:
def N2S(act0,act1,act2,wei0,wei1,wei2):
    aSF = findActMaxMin()
    wSF = findWeiMaxMin()
    
    lfsrlist0 = LFSRlist()
    lfsrlist1 = LFSRlist()
    lfsrlist2 = LFSRlist()
    
    a0 = SNGnumpy(act0/aSF['act0'],lfsrlist0)
    a1 = SNGnumpy(act1/aSF['act1'],lfsrlist1)
    a2 = SNGnumpy(act2/aSF['act2'],lfsrlist2)
    
    w0 = SNGpnumpy(wei0/wSF['wei0'],lfsrlist0)
    w1 = SNGpnumpy(wei1/wSF['wei1'],lfsrlist1)
    w2 = SNGpnumpy(wei2/wSF['wei2'],lfsrlist2)
    
    return a0,a1,a2,w0,w1,w2

In [121]:
def S2N(a0,a1,a2,w0,w1,w2):
    aSF = findActMaxMin()
    wSF = findWeiMaxMin()
    
    sa0 = defSign(a0)
    sa1 = defSign(a1)
    sa2 = defSign(a2)
    sw0 = defSign(w0)
    sw1 = defSign(w1)
    sw2 = defSign(w2)
    

    ca0 = CountOne(a0)/(2**args.bBW) * aSF['act0'] *sa0
    ca1 = CountOne(a1)/(2**args.bBW) * aSF['act1'] *sa1
    ca2 = CountOne(a2)/(2**args.bBW) * aSF['act2'] *sa2
    cw0 = CountOne(w0)/(2**args.bBW) * wSF['wei0'] *sw0
    cw1 = CountOne(w1)/(2**args.bBW) * wSF['wei1'] *sw1
    cw2 = CountOne(w2)/(2**args.bBW) * wSF['wei2'] *sw2

    return ca0,ca1,ca2,cw0,cw1,cw2

In [122]:
a0, a1, a2, w0, w1, w2 = N2S(act0,act1,act2,wei0,wei1,wei2)

In [123]:
ca0, ca1, ca2, cw0, cw1, cw2 = S2N(a0,a1,a2,w0,w1,w2)

In [124]:
print(a0)

[['01000010000011011' '10110101111000000' '01101010000111111'
  '10110101111000000' '10010100111000000' '10010000111000000'
  '01101010000111111' '01000000000011011' '10110101111000000']
 ['10010101111000000' '00000000000000000' '01000000000011001'
  '10111111111100100' '10010100111000000' '10010000111000000'
  '01101010000111111' '10010000111000000' '01101010000111111']
 ['01000000000001001' '01111111100111111' '00000000000001001'
  '01111111000111111' '10111111111100100' '10000000001000000'
  '01000000000011001' '00000000000001001' '01001010000111011']
 ['10111101111100100' '10010101111000000' '01001010000111011'
  '01000000000011011' '01001010000111111' '01000000000001001'
  '01000000000001001' '01001010000111111' '00000000000000000']
 ['10110101111100100' '10110101111000000' '01000010000011011'
  '10110101111000100' '10000000111000000' '10000000111000000'
  '10110101111000100' '10000000001000000' '10111111111100100']
 ['01111111111111111' '01111111110111111' '00000000000000001'
  '

In [125]:
print(act0)
print(ca0)

tensor([[ 1.3203, -1.8813,  2.2693, -1.7178, -1.3540, -1.0659,  2.3939,  1.2291,
         -1.7365],
        [-1.4850,  3.7657,  0.7815, -2.6903, -1.2147, -1.0142,  2.2421, -0.9429,
          2.3232],
        [ 0.7570,  3.3001,  0.5164,  2.9171, -2.7603, -0.3146,  0.9761,  0.3926,
          1.9125],
        [-2.5010, -1.6019,  1.8423,  1.0435,  1.9623,  0.7016,  0.5911,  1.9841,
          0.0142],
        [-2.1889, -1.7319,  1.4073, -1.9775, -0.9190, -0.7879, -1.9521, -0.4280,
         -2.7960],
        [ 3.6417,  3.5225,  0.2619,  2.1692, -0.0894,  2.4622,  0.0818,  1.8878,
          0.8327],
        [-0.6412,  2.8229,  0.8840,  1.9618,  0.8510, -0.8657,  0.6282,  2.1382,
         -0.7442],
        [-1.9318,  0.7751,  3.4735, -0.5305,  0.8575, -0.5334, -0.4971,  1.6270,
          1.1518],
        [-0.8343, -0.4570,  0.0687,  0.5031,  2.7980,  0.7810,  0.0387, -1.6112,
          1.8086]])
tensor([[ 1.1768, -1.8829,  2.1182, -1.8829, -1.4122, -1.1768,  2.1182,  0.9414,
         -1.8829],